### 1. SPARK 환경 설정 & spark 세션 연결

In [1]:
!pip3 install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=99d82a58e2953b29396658afc50a39fb540785a31e6d83a1bf9db41f8eeff680
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from pyspark.sql import SparkSession, Row
from pyspark.sql import types as T
from pyspark.sql import window as W
from pyspark.sql import functions as F

spark = SparkSession.builder \
        .master("local") \
        .appName("Colab") \
        .getOrCreate()

In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv("/content/drive/MyDrive/Python Spark/test1.csv")
df

,Name,age,Experience,Salary
0,Paul,31,10,30000
1,Tom,30,8,25000
2,Sunny,29,4,20000
3,Paul,24,3,20000
4,Ann,21,1,15000
5,Peter,23,2,18000


### 2. 데이터 읽기 & 쓰기

In [5]:
## 데이터 읽기
import pandas as pd
import numpy as np

In [8]:
df = spark.read.csv("/content/drive/MyDrive/Python Spark/test1.csv", header=True)

In [9]:
df.show()

+-----+---+----------+------+
| Name|age|Experience|Salary|
+-----+---+----------+------+
| Paul| 31|        10| 30000|
|  Tom| 30|         8| 25000|
|Sunny| 29|         4| 20000|
| Paul| 24|         3| 20000|
|  Ann| 21|         1| 15000|
|Peter| 23|         2| 18000|
+-----+---+----------+------+



In [10]:
# schema 확인

df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- Experience: string (nullable = true)
 |-- Salary: string (nullable = true)



In [11]:
df.dtypes

[('Name', 'string'),
 ('age', 'string'),
 ('Experience', 'string'),
 ('Salary', 'string')]

In [14]:
# 타입 확인

print(type(df))

<class 'pyspark.sql.dataframe.DataFrame'>


In [17]:
# 다른 방법으로 보기
#df.show() 는 low값이 20개가 default
# df.show(2)
df.show(1,vertical=True, truncate=False)

-RECORD 0-----------
 Name       | Paul  
 age        | 31    
 Experience | 10    
 Salary     | 30000 
only showing top 1 row



In [18]:
# 일부 row만 보기

df.limit(3).show()

+-----+---+----------+------+
| Name|age|Experience|Salary|
+-----+---+----------+------+
| Paul| 31|        10| 30000|
|  Tom| 30|         8| 25000|
|Sunny| 29|         4| 20000|
+-----+---+----------+------+



In [19]:
# 컬럼 확인
df.columns
#강력하다 특히 join같은 거 길어질 때, columns만을 보기 위해서 자겅ㅂ한다

['Name', 'age', 'Experience', 'Salary']

In [20]:
# row count 확인


df.count()

6

In [22]:
# Row 형태로 보기
df.collect()

[Row(Name='Paul', age='31', Experience='10', Salary='30000'),
 Row(Name='Tom', age='30', Experience='8', Salary='25000'),
 Row(Name='Sunny', age='29', Experience='4', Salary='20000'),
 Row(Name='Paul', age='24', Experience='3', Salary='20000'),
 Row(Name='Ann', age='21', Experience='1', Salary='15000'),
 Row(Name='Peter', age='23', Experience='2', Salary='18000')]

In [24]:
df.first()

Row(Name='Paul', age='31', Experience='10', Salary='30000')

In [29]:
first_row = df.first()

first_row.Name, first_row.age

('Paul', '31')

In [34]:
# 컬럼 선택
# df.columns

df.select(['Name', 'age']).show()

#show를 붙여야한다!

+-----+---+
| Name|age|
+-----+---+
| Paul| 31|
|  Tom| 30|
|Sunny| 29|
| Paul| 24|
|  Ann| 21|
|Peter| 23|
+-----+---+



In [39]:
# 컬럼 삭제
df_e = df.drop('Experience')
df_e.show()

+-----+---+------+
| Name|age|Salary|
+-----+---+------+
| Paul| 31| 30000|
|  Tom| 30| 25000|
|Sunny| 29| 20000|
| Paul| 24| 20000|
|  Ann| 21| 15000|
|Peter| 23| 18000|
+-----+---+------+



In [41]:
# 컬럼 추가
df.withColumn('new_age',df.age+10).show()

+-----+---+----------+------+-------+
| Name|age|Experience|Salary|new_age|
+-----+---+----------+------+-------+
| Paul| 31|        10| 30000|   41.0|
|  Tom| 30|         8| 25000|   40.0|
|Sunny| 29|         4| 20000|   39.0|
| Paul| 24|         3| 20000|   34.0|
|  Ann| 21|         1| 15000|   31.0|
|Peter| 23|         2| 18000|   33.0|
+-----+---+----------+------+-------+



In [42]:
df.withColumn('age',df.age**2).show()


+-----+-----+----------+------+
| Name|  age|Experience|Salary|
+-----+-----+----------+------+
| Paul|961.0|        10| 30000|
|  Tom|900.0|         8| 25000|
|Sunny|841.0|         4| 20000|
| Paul|576.0|         3| 20000|
|  Ann|441.0|         1| 15000|
|Peter|529.0|         2| 18000|
+-----+-----+----------+------+



In [44]:
# 컬럼 이름 변경

df.withColumnRenamed('age', 'new_age').show()

+-----+-------+----------+------+
| Name|new_age|Experience|Salary|
+-----+-------+----------+------+
| Paul|     31|        10| 30000|
|  Tom|     30|         8| 25000|
|Sunny|     29|         4| 20000|
| Paul|     24|         3| 20000|
|  Ann|     21|         1| 15000|
|Peter|     23|         2| 18000|
+-----+-------+----------+------+



In [45]:
# 컬럼 타입 변경
df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- Experience: string (nullable = true)
 |-- Salary: string (nullable = true)



In [47]:
# 새로운 dataframe 생성
# cast

df.withColumn('age',  F.col('age').cast(T.IntegerType())).printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: string (nullable = true)
 |-- Salary: string (nullable = true)



In [55]:
# dataframe 쓰기
#SparkSession.createDataFrame(data, schema=None, samplingRatio=None, verifySchema=True

schema = T.StructType(
    [
        T.StructField('aaa', T.StringType(), True),
        T.StructField('bbb', T.StringType(), True),
        T.StructField('ccc', T.StringType(), True)
    ]
)
data_1 = [
    Row('apple', 'banana', 'tomato'),
    Row('apple1', 'banana1', 'tomato1'),
    Row('apple2', 'banana2', 'tomato2')
]

data_2 = [
    Row('apple3', 'banana3', 'tomato3'),
    Row('apple4', 'banana4', 'tomato4'),
    Row('apple5', 'banana5', 'tomato5')
]

df_new = spark.createDataFrame(data_2, schema)

In [56]:
df_new.show()

+------+-------+-------+
|   aaa|    bbb|    ccc|
+------+-------+-------+
|apple3|banana3|tomato3|
|apple4|banana4|tomato4|
|apple5|banana5|tomato5|
+------+-------+-------+



In [ ]:
#dataframe 쓰기 -> write
#df.write.mode().csv('저장될 경로')
df.write.mode('overwrite').csv('/content/drive/MyDrive/')



df.coalesce(1).write.mode('overwrite').csv('/content/drive/MyDrive/')

### 3. 결측치 다루기

In [57]:
# 결측치 drop 하기

df = spark.read.csv("/content/drive/MyDrive/Python Spark/test2.csv", header=True)

df.show()


+-----+----+----------+------+
| Name| age|Experience|Salary|
+-----+----+----------+------+
| Paul|  31|        10| 30000|
| Lucy|  30|         8| 25000|
|Sunny|  29|         4| 20000|
| Paul|  24|         3| 20000|
|  Ann|  21|         1| 15000|
|  Tom|  23|         2| 18000|
| Lucy|NULL|      NULL| 40000|
| NULL|  34|        10| 38000|
| NULL|  36|      NULL|  NULL|
+-----+----+----------+------+



In [58]:
# thresh : null 값이 thresh개 보다 많이 있으면 drop
# df.na.drop().show()

+-----+---+----------+------+
| Name|age|Experience|Salary|
+-----+---+----------+------+
| Paul| 31|        10| 30000|
| Lucy| 30|         8| 25000|
|Sunny| 29|         4| 20000|
| Paul| 24|         3| 20000|
|  Ann| 21|         1| 15000|
|  Tom| 23|         2| 18000|
+-----+---+----------+------+



In [60]:
# any : null값이 하나라도 있으면 제거
# all : 모든 row가 null인 row만 제거
df.dropna().show()

+-----+---+----------+------+
| Name|age|Experience|Salary|
+-----+---+----------+------+
| Paul| 31|        10| 30000|
| Lucy| 30|         8| 25000|
|Sunny| 29|         4| 20000|
| Paul| 24|         3| 20000|
|  Ann| 21|         1| 15000|
|  Tom| 23|         2| 18000|
+-----+---+----------+------+



In [63]:
# 특정 컬럼에 대해서
# thresh : 정상값이 thresh개 보다 적게 있는 row를 drop한다!
df.dropna(thresh=3).show()

+-----+---+----------+------+
| Name|age|Experience|Salary|
+-----+---+----------+------+
| Paul| 31|        10| 30000|
| Lucy| 30|         8| 25000|
|Sunny| 29|         4| 20000|
| Paul| 24|         3| 20000|
|  Ann| 21|         1| 15000|
|  Tom| 23|         2| 18000|
| NULL| 34|        10| 38000|
+-----+---+----------+------+



In [64]:
#how any
df.na.drop(how='any').show()
#하나라도 null값이 있으면 컷


+-----+---+----------+------+
| Name|age|Experience|Salary|
+-----+---+----------+------+
| Paul| 31|        10| 30000|
| Lucy| 30|         8| 25000|
|Sunny| 29|         4| 20000|
| Paul| 24|         3| 20000|
|  Ann| 21|         1| 15000|
|  Tom| 23|         2| 18000|
+-----+---+----------+------+



In [65]:
df.na.drop(how='all').show()#
#모든 컬럼에 속한 데이터 값들이 null 값이면 다 컷

+-----+----+----------+------+
| Name| age|Experience|Salary|
+-----+----+----------+------+
| Paul|  31|        10| 30000|
| Lucy|  30|         8| 25000|
|Sunny|  29|         4| 20000|
| Paul|  24|         3| 20000|
|  Ann|  21|         1| 15000|
|  Tom|  23|         2| 18000|
| Lucy|NULL|      NULL| 40000|
| NULL|  34|        10| 38000|
| NULL|  36|      NULL|  NULL|
+-----+----+----------+------+



In [71]:
#특정 컬럼에 대해서
#lucy 지우기
df.na.drop(how = 'any', subset = ['age', 'Experience']).show()

+-----+---+----------+------+
| Name|age|Experience|Salary|
+-----+---+----------+------+
| Paul| 31|        10| 30000|
| Lucy| 30|         8| 25000|
|Sunny| 29|         4| 20000|
| Paul| 24|         3| 20000|
|  Ann| 21|         1| 15000|
|  Tom| 23|         2| 18000|
| NULL| 34|        10| 38000|
+-----+---+----------+------+



In [73]:
# 결측치 채우기, value가 해당 컬럼의 type과 맞아야 한다.

df.na.fill(value = '가나다', subset = ['Name', 'Salary']).show()

+------+----+----------+------+
|  Name| age|Experience|Salary|
+------+----+----------+------+
|  Paul|  31|        10| 30000|
|  Lucy|  30|         8| 25000|
| Sunny|  29|         4| 20000|
|  Paul|  24|         3| 20000|
|   Ann|  21|         1| 15000|
|   Tom|  23|         2| 18000|
|  Lucy|NULL|      NULL| 40000|
|가나다|  34|        10| 38000|
|가나다|  36|      NULL|가나다|
+------+----+----------+------+



In [74]:
df.fillna(value = '컴퓨터', subset = ['age', 'Experience', 'Salary']).show()


+-----+------+----------+------+
| Name|   age|Experience|Salary|
+-----+------+----------+------+
| Paul|    31|        10| 30000|
| Lucy|    30|         8| 25000|
|Sunny|    29|         4| 20000|
| Paul|    24|         3| 20000|
|  Ann|    21|         1| 15000|
|  Tom|    23|         2| 18000|
| Lucy|컴퓨터|    컴퓨터| 40000|
| NULL|    34|        10| 38000|
| NULL|    36|    컴퓨터|컴퓨터|
+-----+------+----------+------+



In [75]:
df_3 = spark.read.csv("/content/drive/MyDrive/Python Spark/test3.csv", header=True)
df_3.show()

+-----+------------+------+
| Name| Departments|salary|
+-----+------------+------+
|  Ann|Data Science| 10000|
|  Ann|         IOT|  5000|
|  Tom|    Big Data|  4000|
|  Ann|    Big Data|  4000|
|  Tom|Data Science|  3000|
|Peter|Data Science| 20000|
|Peter|         IOT| 10000|
|Peter|    Big Data|  5000|
|Sunny|Data Science| 10000|
|Sunny|    Big Data|  2000|
+-----+------------+------+



In [76]:
df_3.withColumn('Salary',df_3.salary+50000).show()


+-----+------------+-------+
| Name| Departments| Salary|
+-----+------------+-------+
|  Ann|Data Science|60000.0|
|  Ann|         IOT|55000.0|
|  Tom|    Big Data|54000.0|
|  Ann|    Big Data|54000.0|
|  Tom|Data Science|53000.0|
|Peter|Data Science|70000.0|
|Peter|         IOT|60000.0|
|Peter|    Big Data|55000.0|
|Sunny|Data Science|60000.0|
|Sunny|    Big Data|52000.0|
+-----+------------+-------+



In [77]:
df_3.withColumn('in',df_3.salary+50000).show()


+-----+------------+------+-------+
| Name| Departments|salary|     in|
+-----+------------+------+-------+
|  Ann|Data Science| 10000|60000.0|
|  Ann|         IOT|  5000|55000.0|
|  Tom|    Big Data|  4000|54000.0|
|  Ann|    Big Data|  4000|54000.0|
|  Tom|Data Science|  3000|53000.0|
|Peter|Data Science| 20000|70000.0|
|Peter|         IOT| 10000|60000.0|
|Peter|    Big Data|  5000|55000.0|
|Sunny|Data Science| 10000|60000.0|
|Sunny|    Big Data|  2000|52000.0|
+-----+------------+------+-------+



+-----+------------+------+
| Name| Departments|salary|
+-----+------------+------+
|  Ann|Data Science| 10000|
|  Ann|         IOT|  5000|
|  Tom|    Big Data|  4000|
|  Ann|    Big Data|  4000|
|  Tom|Data Science|  3000|
|Peter|Data Science| 20000|
|Peter|         IOT| 10000|
|Peter|    Big Data|  5000|
|Sunny|Data Science| 10000|
|Sunny|    Big Data|  2000|
+-----+------------+------+



### 4. filter

In [89]:
# 단일 필터
item = spark.read.csv('/content/drive/MyDrive/Python Spark/item_his.csv', header=True)

item.show(20)

+-----+-----+-------+--------+----------+--------------+--------+-----+
|  idx|   lv|proc_ym|proc_ymd|  codename|   mascodename|payprice|price|
+-----+-----+-------+--------+----------+--------------+--------+-----+
|53687|175.0| 202305|20230508|  액세서리|아바타파츠구분|      50|   50|
|53687|175.0| 202305|20230508|상태메시지|  기타파츠구분|      50|   50|
|20163|161.0| 202304|20230408|  액세서리|아바타파츠구분|      50|   50|
|88572| 14.0| 202304|20230408|상태메시지|  기타파츠구분|      50|   50|
|88572| 14.0| 202304|20230408|상태메시지|  기타파츠구분|      50|   50|
|26112|130.0| 202304|20230405|  액세서리|아바타파츠구분|      50|   50|
|26112|130.0| 202304|20230405|      헤어|아바타파츠구분|     150|  150|
|49541|170.0| 202306|20230630|      얼굴|아바타파츠구분|     150|  150|
|49541|170.0| 202306|20230630|      신발|아바타파츠구분|     150|  150|
|49541|170.0| 202306|20230630|상태메시지|  기타파츠구분|      50|   50|
|49541|170.0| 202306|20230630|      헤어|아바타파츠구분|     150|  150|
|49541|170.0| 202306|20230630|      헤어|아바타파츠구분|     150|  150|
|37822| 39.0| 202306|20230608|      헤어|아바타

In [85]:
item.filter(F.col('codename') == '액세서리').show(5)

+-----+-----+-------+--------+--------+--------------+--------+-----+
|  idx|   lv|proc_ym|proc_ymd|codename|   mascodename|payprice|price|
+-----+-----+-------+--------+--------+--------------+--------+-----+
|53687|175.0| 202305|20230508|액세서리|아바타파츠구분|      50|   50|
|20163|161.0| 202304|20230408|액세서리|아바타파츠구분|      50|   50|
|26112|130.0| 202304|20230405|액세서리|아바타파츠구분|      50|   50|
|76257|190.0| 202306|20230608|액세서리|아바타파츠구분|      25|   25|
|76257|190.0| 202306|20230608|액세서리|아바타파츠구분|      25|   25|
+-----+-----+-------+--------+--------+--------------+--------+-----+
only showing top 5 rows



In [90]:
item = item.withColumn('payprice',  F.col('payprice').cast(T.IntegerType()))

item.printSchema()

root
 |-- idx: string (nullable = true)
 |-- lv: string (nullable = true)
 |-- proc_ym: string (nullable = true)
 |-- proc_ymd: string (nullable = true)
 |-- codename: string (nullable = true)
 |-- mascodename: string (nullable = true)
 |-- payprice: integer (nullable = true)
 |-- price: string (nullable = true)



In [91]:
item.filter(F.col('payprice') < 50).show()

+-----+-----+-------+--------+--------+--------------+--------+-----+
|  idx|   lv|proc_ym|proc_ymd|codename|   mascodename|payprice|price|
+-----+-----+-------+--------+--------+--------------+--------+-----+
|76257|190.0| 202306|20230608|액세서리|아바타파츠구분|      25|   25|
|76257|190.0| 202306|20230608|액세서리|아바타파츠구분|      25|   25|
|32960|151.0| 202305|20230510|액세서리|아바타파츠구분|      25|   25|
|61823|151.0| 202304|20230418|액세서리|아바타파츠구분|      25|   25|
|77389|118.0| 202304|20230421|액세서리|아바타파츠구분|      25|   25|
|13456|158.0| 202304|20230421|액세서리|아바타파츠구분|      25|   25|
|74784|156.0| 202306|20230619|액세서리|아바타파츠구분|      25|   25|
|86844|152.0| 202306|20230609|액세서리|아바타파츠구분|      25|   25|
|52543|152.0| 202304|20230424|액세서리|아바타파츠구분|      25|   25|
|43710|174.0| 202305|20230520|액세서리|아바타파츠구분|      25|   25|
|13841|170.0| 202304|20230418|액세서리|아바타파츠구분|      25|   25|
|13841|170.0| 202304|20230418|액세서리|아바타파츠구분|      25|   25|
|13841|170.0| 202304|20230418|액세서리|아바타파츠구분|      25|   25|
|13841|170.0| 202304|20

In [92]:
# 다중 필터

item.filter(F.col('codename') == '액세서리').filter(F.col('payprice') < 50).show()

+-----+-----+-------+--------+--------+--------------+--------+-----+
|  idx|   lv|proc_ym|proc_ymd|codename|   mascodename|payprice|price|
+-----+-----+-------+--------+--------+--------------+--------+-----+
|76257|190.0| 202306|20230608|액세서리|아바타파츠구분|      25|   25|
|76257|190.0| 202306|20230608|액세서리|아바타파츠구분|      25|   25|
|32960|151.0| 202305|20230510|액세서리|아바타파츠구분|      25|   25|
|61823|151.0| 202304|20230418|액세서리|아바타파츠구분|      25|   25|
|77389|118.0| 202304|20230421|액세서리|아바타파츠구분|      25|   25|
|13456|158.0| 202304|20230421|액세서리|아바타파츠구분|      25|   25|
|74784|156.0| 202306|20230619|액세서리|아바타파츠구분|      25|   25|
|86844|152.0| 202306|20230609|액세서리|아바타파츠구분|      25|   25|
|52543|152.0| 202304|20230424|액세서리|아바타파츠구분|      25|   25|
|43710|174.0| 202305|20230520|액세서리|아바타파츠구분|      25|   25|
|13841|170.0| 202304|20230418|액세서리|아바타파츠구분|      25|   25|
|13841|170.0| 202304|20230418|액세서리|아바타파츠구분|      25|   25|
|13841|170.0| 202304|20230418|액세서리|아바타파츠구분|      25|   25|
|13841|170.0| 202304|20

In [96]:
#and 조건과 or 조건 모두 가능하다! & or |
item.filter((F.col('codename') == '액세서리') | (F.col('payprice') < 50)).show()


+-----+-----+-------+--------+--------+--------------+--------+-----+
|  idx|   lv|proc_ym|proc_ymd|codename|   mascodename|payprice|price|
+-----+-----+-------+--------+--------+--------------+--------+-----+
|53687|175.0| 202305|20230508|액세서리|아바타파츠구분|      50|   50|
|20163|161.0| 202304|20230408|액세서리|아바타파츠구분|      50|   50|
|26112|130.0| 202304|20230405|액세서리|아바타파츠구분|      50|   50|
|76257|190.0| 202306|20230608|액세서리|아바타파츠구분|      25|   25|
|76257|190.0| 202306|20230608|액세서리|아바타파츠구분|      25|   25|
|67188|175.0| 202306|20230612|액세서리|아바타파츠구분|      50|   50|
|11987|159.0| 202304|20230404|액세서리|아바타파츠구분|      50|   50|
|48378|144.0| 202304|20230412|액세서리|아바타파츠구분|      50|   50|
|32960|151.0| 202305|20230510|액세서리|아바타파츠구분|      25|   25|
|61823|151.0| 202304|20230418|액세서리|아바타파츠구분|      25|   25|
|84515| 83.0| 202304|20230425|액세서리|아바타파츠구분|      50|   50|
|53306|130.0| 202304|20230410|액세서리|아바타파츠구분|      50|   50|
|77389|118.0| 202304|20230421|액세서리|아바타파츠구분|      25|   25|
|13456|158.0| 202304|20

In [101]:
# 필터 반대 조건

# item.filter(F.col('codename') != '액세서리').show()

item.filter(~(F.col('codename') != '액세서리')).show()



+-----+-----+-------+--------+--------+--------------+--------+-----+
|  idx|   lv|proc_ym|proc_ymd|codename|   mascodename|payprice|price|
+-----+-----+-------+--------+--------+--------------+--------+-----+
|53687|175.0| 202305|20230508|액세서리|아바타파츠구분|      50|   50|
|20163|161.0| 202304|20230408|액세서리|아바타파츠구분|      50|   50|
|26112|130.0| 202304|20230405|액세서리|아바타파츠구분|      50|   50|
|76257|190.0| 202306|20230608|액세서리|아바타파츠구분|      25|   25|
|76257|190.0| 202306|20230608|액세서리|아바타파츠구분|      25|   25|
|67188|175.0| 202306|20230612|액세서리|아바타파츠구분|      50|   50|
|11987|159.0| 202304|20230404|액세서리|아바타파츠구분|      50|   50|
|48378|144.0| 202304|20230412|액세서리|아바타파츠구분|      50|   50|
|32960|151.0| 202305|20230510|액세서리|아바타파츠구분|      25|   25|
|61823|151.0| 202304|20230418|액세서리|아바타파츠구분|      25|   25|
|84515| 83.0| 202304|20230425|액세서리|아바타파츠구분|      50|   50|
|53306|130.0| 202304|20230410|액세서리|아바타파츠구분|      50|   50|
|77389|118.0| 202304|20230421|액세서리|아바타파츠구분|      25|   25|
|13456|158.0| 202304|20

In [103]:

# 중복 제거 -> distinct, dropDuplicates, drop_Duplicates
#

# item.count() # 109545


# item.distinct().count() -> #83064





83064

In [104]:
# item.dropDuplicates().count() -> #distinct와 동일하다

item.dropDuplicates(subset = ['codename']).count()

8

In [105]:
item.dropDuplicates(subset = ['codename']).show()

+-----+-----+-------+--------+----------+--------------+--------+-----+
|  idx|   lv|proc_ym|proc_ymd|  codename|   mascodename|payprice|price|
+-----+-----+-------+--------+----------+--------------+--------+-----+
|67188|175.0| 202306|20230612|      상의|아바타파츠구분|     200|  200|
|53687|175.0| 202305|20230508|상태메시지|  기타파츠구분|      50|   50|
|49541|170.0| 202306|20230630|      신발|아바타파츠구분|     150|  150|
|53687|175.0| 202305|20230508|  액세서리|아바타파츠구분|      50|   50|
|49541|170.0| 202306|20230630|      얼굴|아바타파츠구분|     150|  150|
|92027| NULL| 202305|20230505|    코스튬|아바타파츠구분|     300|  300|
|84801|112.0| 202306|20230627|      하의|아바타파츠구분|     250|  250|
|26112|130.0| 202304|20230405|      헤어|아바타파츠구분|     150|  150|
+-----+-----+-------+--------+----------+--------------+--------+-----+



In [107]:
item.dropDuplicates(subset = ['codename', 'payprice']).show()

+-----+-----+-------+--------+----------+--------------+--------+-----+
|  idx|   lv|proc_ym|proc_ymd|  codename|   mascodename|payprice|price|
+-----+-----+-------+--------+----------+--------------+--------+-----+
|77041|169.0| 202306|20230628|      상의|아바타파츠구분|     150|  150|
|67188|175.0| 202306|20230612|      상의|아바타파츠구분|     200|  200|
|53687|175.0| 202305|20230508|상태메시지|  기타파츠구분|      50|   50|
|81788| 15.0| 202304|20230420|상태메시지|  기타파츠구분|     100|  100|
|67188|175.0| 202306|20230612|      신발|아바타파츠구분|      60|   60|
|49541|170.0| 202306|20230630|      신발|아바타파츠구분|     150|  150|
|76257|190.0| 202306|20230608|  액세서리|아바타파츠구분|      25|   25|
|53687|175.0| 202305|20230508|  액세서리|아바타파츠구분|      50|   50|
|49541|170.0| 202306|20230630|      얼굴|아바타파츠구분|     150|  150|
|92027| NULL| 202305|20230505|    코스튬|아바타파츠구분|     300|  300|
|84801|112.0| 202306|20230627|      하의|아바타파츠구분|     250|  250|
|26112|130.0| 202304|20230405|      헤어|아바타파츠구분|     150|  150|
+-----+-----+-------+--------+-------

In [112]:
#  특정 칼럼 특정 데이터 포함 여부 확인 (isin)


item.filter(F.col('codename').isin(['헤어', '얼굴', '신발'])).show(5)

+-----+-----+-------+--------+--------+--------------+--------+-----+
|  idx|   lv|proc_ym|proc_ymd|codename|   mascodename|payprice|price|
+-----+-----+-------+--------+--------+--------------+--------+-----+
|26112|130.0| 202304|20230405|    헤어|아바타파츠구분|     150|  150|
|49541|170.0| 202306|20230630|    얼굴|아바타파츠구분|     150|  150|
|49541|170.0| 202306|20230630|    신발|아바타파츠구분|     150|  150|
|49541|170.0| 202306|20230630|    헤어|아바타파츠구분|     150|  150|
|49541|170.0| 202306|20230630|    헤어|아바타파츠구분|     150|  150|
+-----+-----+-------+--------+--------+--------------+--------+-----+
only showing top 5 rows



In [116]:
# Null값 확인 isnull(), isNotNull()


# df.filter(F.col('Name').isNull()).show()

df.filter(F.col('Name').isNotNull()).show()

+-----+----+----------+------+
| Name| age|Experience|Salary|
+-----+----+----------+------+
| Paul|  31|        10| 30000|
| Lucy|  30|         8| 25000|
|Sunny|  29|         4| 20000|
| Paul|  24|         3| 20000|
|  Ann|  21|         1| 15000|
|  Tom|  23|         2| 18000|
| Lucy|NULL|      NULL| 40000|
+-----+----+----------+------+



In [123]:
# 비슷한 형태 확인 (like)

item.filter(F.col('mascodename'). like('아바타%')).show()

+-----+-----+-------+--------+--------+--------------+--------+-----+
|  idx|   lv|proc_ym|proc_ymd|codename|   mascodename|payprice|price|
+-----+-----+-------+--------+--------+--------------+--------+-----+
|53687|175.0| 202305|20230508|액세서리|아바타파츠구분|      50|   50|
|20163|161.0| 202304|20230408|액세서리|아바타파츠구분|      50|   50|
|26112|130.0| 202304|20230405|액세서리|아바타파츠구분|      50|   50|
|26112|130.0| 202304|20230405|    헤어|아바타파츠구분|     150|  150|
|49541|170.0| 202306|20230630|    얼굴|아바타파츠구분|     150|  150|
|49541|170.0| 202306|20230630|    신발|아바타파츠구분|     150|  150|
|49541|170.0| 202306|20230630|    헤어|아바타파츠구분|     150|  150|
|49541|170.0| 202306|20230630|    헤어|아바타파츠구분|     150|  150|
|37822| 39.0| 202306|20230608|    헤어|아바타파츠구분|     150|  150|
|76257|190.0| 202306|20230608|액세서리|아바타파츠구분|      25|   25|
|76257|190.0| 202306|20230608|액세서리|아바타파츠구분|      25|   25|
|84801|112.0| 202306|20230627|    헤어|아바타파츠구분|     150|  150|
|84801|112.0| 202306|20230627|    하의|아바타파츠구분|     250|  250|
|84801|

In [129]:
# When 조건문



df.withColumn('new_age', F.when(F.col('age').isNull(), '-').otherwise(F.col('Salary') ** 2)).show()




+-----+----+----------+------+-------+
| Name| age|Experience|Salary|new_age|
+-----+----+----------+------+-------+
| Paul|  31|        10| 30000|  9.0E8|
| Lucy|  30|         8| 25000| 6.25E8|
|Sunny|  29|         4| 20000|  4.0E8|
| Paul|  24|         3| 20000|  4.0E8|
|  Ann|  21|         1| 15000| 2.25E8|
|  Tom|  23|         2| 18000| 3.24E8|
| Lucy|NULL|      NULL| 40000|      -|
| NULL|  34|        10| 38000|1.444E9|
| NULL|  36|      NULL|  NULL|   NULL|
+-----+----+----------+------+-------+



In [130]:
item.filter(F.col('payprice').between(100,200)).show()

+-----+-----+-------+--------+--------+--------------+--------+-----+
|  idx|   lv|proc_ym|proc_ymd|codename|   mascodename|payprice|price|
+-----+-----+-------+--------+--------+--------------+--------+-----+
|26112|130.0| 202304|20230405|    헤어|아바타파츠구분|     150|  150|
|49541|170.0| 202306|20230630|    얼굴|아바타파츠구분|     150|  150|
|49541|170.0| 202306|20230630|    신발|아바타파츠구분|     150|  150|
|49541|170.0| 202306|20230630|    헤어|아바타파츠구분|     150|  150|
|49541|170.0| 202306|20230630|    헤어|아바타파츠구분|     150|  150|
|37822| 39.0| 202306|20230608|    헤어|아바타파츠구분|     150|  150|
|84801|112.0| 202306|20230627|    헤어|아바타파츠구분|     150|  150|
|84801|112.0| 202306|20230627|    얼굴|아바타파츠구분|     150|  150|
|67188|175.0| 202306|20230612|    얼굴|아바타파츠구분|     150|  150|
|67188|175.0| 202306|20230612|    상의|아바타파츠구분|     200|  200|
|67188|175.0| 202306|20230612|    헤어|아바타파츠구분|     150|  150|
| 3663|192.0| 202306|20230612|    얼굴|아바타파츠구분|     150|  150|
| 6533|164.0| 202305|20230511|    헤어|아바타파츠구분|     150|  15

### 5. aggregation + groupby

In [132]:
# group by 함수

item.groupby(F.col('payprice')).sum().show()

+--------+-------------+
|payprice|sum(payprice)|
+--------+-------------+
|     300|      1409100|
|     250|      2299500|
|     100|       331500|
|      50|      1164600|
|      25|        60025|
|      60|       869220|
|     200|      2178600|
|     150|      6189300|
+--------+-------------+



In [134]:
# group by + aggregation 함수
item.groupby(F.col('codename')).count().show()

item.groupby('codename').mean().show()

+----------+-----+
|  codename|count|
+----------+-----+
|      신발|17245|
|      하의| 9198|
|  액세서리|11570|
|      얼굴|14075|
|      헤어|24011|
|    코스튬| 4697|
|상태메시지|17438|
|      상의|11311|
+----------+-----+

+----------+------------------+
|  codename|     avg(payprice)|
+----------+------------------+
|      신발| 74.39373731516382|
|      하의|             250.0|
|  액세서리| 44.81201382886776|
|      얼굴|             150.0|
|      헤어|             150.0|
|    코스튬|             300.0|
|상태메시지| 59.50510379630692|
|      상의|198.15224118115108|
+----------+------------------+



In [136]:
item.groupby("codename").agg(F.count('codename').alias('countcount')).show()

+----------+----------+
|  codename|countcount|
+----------+----------+
|      신발|     17245|
|      하의|      9198|
|  액세서리|     11570|
|      얼굴|     14075|
|      헤어|     24011|
|    코스튬|      4697|
|상태메시지|     17438|
|      상의|     11311|
+----------+----------+



In [137]:
item.groupby('codename').agg(F.mean(F.col('payprice')).alias('codename_mean')).show()

+----------+------------------+
|  codename|     codename_mean|
+----------+------------------+
|      신발| 74.39373731516382|
|      하의|             250.0|
|  액세서리| 44.81201382886776|
|      얼굴|             150.0|
|      헤어|             150.0|
|    코스튬|             300.0|
|상태메시지| 59.50510379630692|
|      상의|198.15224118115108|
+----------+------------------+



In [140]:
item.groupby('codename').agg(
    F.mean(F.col('payprice')).alias('codename_mean'), F.max(F.col('payprice').alias('codename_max')),
    F.min(F.col('payprice')).alias('codename_min'),
    F.round(F.mean(F.col('payprice')), 2).alias('codename_round')
).show()

+----------+------------------+-----------------------------+------------+--------------+
|  codename|     codename_mean|max(payprice AS codename_max)|codename_min|codename_round|
+----------+------------------+-----------------------------+------------+--------------+
|      신발| 74.39373731516382|                          150|          60|         74.39|
|      하의|             250.0|                          250|         250|         250.0|
|  액세서리| 44.81201382886776|                           50|          25|         44.81|
|      얼굴|             150.0|                          150|         150|         150.0|
|      헤어|             150.0|                          150|         150|         150.0|
|    코스튬|             300.0|                          300|         300|         300.0|
|상태메시지| 59.50510379630692|                          100|          50|         59.51|
|      상의|198.15224118115108|                          200|         150|        198.15|
+----------+------------------+-

In [141]:
item.show(5)

+-----+-----+-------+--------+----------+--------------+--------+-----+
|  idx|   lv|proc_ym|proc_ymd|  codename|   mascodename|payprice|price|
+-----+-----+-------+--------+----------+--------------+--------+-----+
|53687|175.0| 202305|20230508|  액세서리|아바타파츠구분|      50|   50|
|53687|175.0| 202305|20230508|상태메시지|  기타파츠구분|      50|   50|
|20163|161.0| 202304|20230408|  액세서리|아바타파츠구분|      50|   50|
|88572| 14.0| 202304|20230408|상태메시지|  기타파츠구분|      50|   50|
|88572| 14.0| 202304|20230408|상태메시지|  기타파츠구분|      50|   50|
+-----+-----+-------+--------+----------+--------------+--------+-----+
only showing top 5 rows



In [146]:
item.groupby('codename').agg(
    F.collect_list(F.col('payprice')).alias('codename_list'),
    # F.collect_set(F.col('payprcie')).alias('codename_set')
).show()
#중복포함

+----------+--------------------+
|  codename|       codename_list|
+----------+--------------------+
|      신발|[150, 60, 60, 60,...|
|      하의|[250, 250, 250, 2...|
|  액세서리|[50, 50, 50, 25, ...|
|      얼굴|[150, 150, 150, 1...|
|      헤어|[150, 150, 150, 1...|
|    코스튬|[300, 300, 300, 3...|
|상태메시지|[50, 50, 50, 50, ...|
|      상의|[200, 200, 200, 2...|
+----------+--------------------+



### 6. order by

In [148]:
df.printSchema

<bound method DataFrame.printSchema of DataFrame[Name: string, age: string, Experience: string, Salary: string]>

In [149]:
# 1) 기본

# df.show()
df = df.withColumn('Experience', F.col('Experience').cast(T.IntegerType()))

df.printSchema()


root
 |-- Name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: string (nullable = true)



In [150]:
# 2) 어려개
df.orderBy('Experience').show()

+-----+----+----------+------+
| Name| age|Experience|Salary|
+-----+----+----------+------+
| Lucy|NULL|      NULL| 40000|
| NULL|  36|      NULL|  NULL|
|  Ann|  21|         1| 15000|
|  Tom|  23|         2| 18000|
| Paul|  24|         3| 20000|
|Sunny|  29|         4| 20000|
| Lucy|  30|         8| 25000|
| Paul|  31|        10| 30000|
| NULL|  34|        10| 38000|
+-----+----+----------+------+



In [151]:
# 3) 여러개 + 정렬 순서 반대
df = df.withColumn('age', F.col('age').cast(T.IntegerType()))
df = df.withColumn('Salary', F.col('Salary').cast(T.IntegerType()))

df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [153]:
df.orderBy('Salary').show()

+-----+----+----------+------+
| Name| age|Experience|Salary|
+-----+----+----------+------+
| NULL|  36|      NULL|  NULL|
|  Ann|  21|         1| 15000|
|  Tom|  23|         2| 18000|
|Sunny|  29|         4| 20000|
| Paul|  24|         3| 20000|
| Lucy|  30|         8| 25000|
| Paul|  31|        10| 30000|
| NULL|  34|        10| 38000|
| Lucy|NULL|      NULL| 40000|
+-----+----+----------+------+



In [155]:
df.orderBy('age', 'Experience').show()

+-----+----+----------+------+
| Name| age|Experience|Salary|
+-----+----+----------+------+
| Lucy|NULL|      NULL| 40000|
|  Ann|  21|         1| 15000|
|  Tom|  23|         2| 18000|
| Paul|  24|         3| 20000|
|Sunny|  29|         4| 20000|
| Lucy|  30|         8| 25000|
| Paul|  31|        10| 30000|
| NULL|  34|        10| 38000|
| NULL|  36|      NULL|  NULL|
+-----+----+----------+------+



In [157]:
df.orderBy('age', ascending=False).show()
#내림차슌!

+-----+----+----------+------+
| Name| age|Experience|Salary|
+-----+----+----------+------+
| NULL|  36|      NULL|  NULL|
| NULL|  34|        10| 38000|
| Paul|  31|        10| 30000|
| Lucy|  30|         8| 25000|
|Sunny|  29|         4| 20000|
| Paul|  24|         3| 20000|
|  Tom|  23|         2| 18000|
|  Ann|  21|         1| 15000|
| Lucy|NULL|      NULL| 40000|
+-----+----+----------+------+



In [158]:
df.orderBy(F.col('age').desc()).show() #컬럼에 대해서 직접적인

+-----+----+----------+------+
| Name| age|Experience|Salary|
+-----+----+----------+------+
| NULL|  36|      NULL|  NULL|
| NULL|  34|        10| 38000|
| Paul|  31|        10| 30000|
| Lucy|  30|         8| 25000|
|Sunny|  29|         4| 20000|
| Paul|  24|         3| 20000|
|  Tom|  23|         2| 18000|
|  Ann|  21|         1| 15000|
| Lucy|NULL|      NULL| 40000|
+-----+----+----------+------+



In [159]:
df.orderBy(F.col('age'), F.col('Experience').desc()).show()

+-----+----+----------+------+
| Name| age|Experience|Salary|
+-----+----+----------+------+
| Lucy|NULL|      NULL| 40000|
|  Ann|  21|         1| 15000|
|  Tom|  23|         2| 18000|
| Paul|  24|         3| 20000|
|Sunny|  29|         4| 20000|
| Lucy|  30|         8| 25000|
| Paul|  31|        10| 30000|
| NULL|  34|        10| 38000|
| NULL|  36|      NULL|  NULL|
+-----+----+----------+------+



### 7. Join & Union

In [177]:
df2 = spark.read.csv('/content/drive/MyDrive/Python Spark/test2.csv', header=True)

df2.show()

+-----+----+----------+------+
| Name| age|Experience|Salary|
+-----+----+----------+------+
| Paul|  31|        10| 30000|
| Lucy|  30|         8| 25000|
|Sunny|  29|         4| 20000|
| Paul|  24|         3| 20000|
|  Ann|  21|         1| 15000|
|  Tom|  23|         2| 18000|
| Lucy|NULL|      NULL| 40000|
| NULL|  34|        10| 38000|
| NULL|  36|      NULL|  NULL|
+-----+----+----------+------+



In [160]:
# 1) 이름이 다른 컬럼들 끼리의 Join
df3 = spark.read.csv('/content/drive/MyDrive/Python Spark/test3.csv', header=True)

df3.show()

+-----+------------+------+
| Name| Departments|salary|
+-----+------------+------+
|  Ann|Data Science| 10000|
|  Ann|         IOT|  5000|
|  Tom|    Big Data|  4000|
|  Ann|    Big Data|  4000|
|  Tom|Data Science|  3000|
|Peter|Data Science| 20000|
|Peter|         IOT| 10000|
|Peter|    Big Data|  5000|
|Sunny|Data Science| 10000|
|Sunny|    Big Data|  2000|
+-----+------------+------+



In [165]:
#1) 이름이 다른 컬럼들끼리의 join

df2 = df2.withColumnRenamed('new_name', 'new_Name')
df2.show()

+--------+----+----------+------+
|new_Name| age|Experience|Salary|
+--------+----+----------+------+
|    Paul|  31|        10| 30000|
|    Lucy|  30|         8| 25000|
|   Sunny|  29|         4| 20000|
|    Paul|  24|         3| 20000|
|     Ann|  21|         1| 15000|
|     Tom|  23|         2| 18000|
|    Lucy|NULL|      NULL| 40000|
|    NULL|  34|        10| 38000|
|    NULL|  36|      NULL|  NULL|
+--------+----+----------+------+



In [169]:
#df2.new_name <-> df3.name

df2.join(df3, df2.new_Name == df3.Name, 'outer').show()


#inner : default, left, right, outer .. 조인등을 쓴다.

#df2가 먼저 나왔으니, df2를 기준으로 df3를 left outer join

+--------+----+----------+------+-----+------------+------+
|new_Name| age|Experience|Salary| Name| Departments|salary|
+--------+----+----------+------+-----+------------+------+
|    NULL|  34|        10| 38000| NULL|        NULL|  NULL|
|    NULL|  36|      NULL|  NULL| NULL|        NULL|  NULL|
|     Ann|  21|         1| 15000|  Ann|Data Science| 10000|
|     Ann|  21|         1| 15000|  Ann|         IOT|  5000|
|     Ann|  21|         1| 15000|  Ann|    Big Data|  4000|
|    Lucy|  30|         8| 25000| NULL|        NULL|  NULL|
|    Lucy|NULL|      NULL| 40000| NULL|        NULL|  NULL|
|    Paul|  31|        10| 30000| NULL|        NULL|  NULL|
|    Paul|  24|         3| 20000| NULL|        NULL|  NULL|
|    NULL|NULL|      NULL|  NULL|Peter|Data Science| 20000|
|    NULL|NULL|      NULL|  NULL|Peter|         IOT| 10000|
|    NULL|NULL|      NULL|  NULL|Peter|    Big Data|  5000|
|   Sunny|  29|         4| 20000|Sunny|Data Science| 10000|
|   Sunny|  29|         4| 20000|Sunny| 

In [178]:
# 2) 이름이 동일한 컬럼들 끼리의 join
# df2.join(df3, df2.Name == df3.Name).show()
df2.join(df3, ['Name']).show()


+-----+---+----------+------+------------+------+
| Name|age|Experience|Salary| Departments|salary|
+-----+---+----------+------+------------+------+
|  Ann| 21|         1| 15000|Data Science| 10000|
|  Ann| 21|         1| 15000|         IOT|  5000|
|  Tom| 23|         2| 18000|    Big Data|  4000|
|  Ann| 21|         1| 15000|    Big Data|  4000|
|  Tom| 23|         2| 18000|Data Science|  3000|
|Sunny| 29|         4| 20000|Data Science| 10000|
|Sunny| 29|         4| 20000|    Big Data|  2000|
+-----+---+----------+------+------------+------+



In [ ]:
# 3) 여러 컬럼들 끼리 join


df2.join(df3, (df2.Name == df3.Name) & (df2.Salary == df3.salary)).show()

In [179]:
df2 = df2.withColumnRenamed('Salary', 'salary')
df2.show()

+-----+----+----------+------+
| Name| age|Experience|salary|
+-----+----+----------+------+
| Paul|  31|        10| 30000|
| Lucy|  30|         8| 25000|
|Sunny|  29|         4| 20000|
| Paul|  24|         3| 20000|
|  Ann|  21|         1| 15000|
|  Tom|  23|         2| 18000|
| Lucy|NULL|      NULL| 40000|
| NULL|  34|        10| 38000|
| NULL|  36|      NULL|  NULL|
+-----+----+----------+------+



In [180]:
# 4) 이름이 동일한 여러 컬럼들 끼리 join
df2.join(df3, ['Name', 'salary']).show()

+----+------+---+----------+-----------+
|Name|salary|age|Experience|Departments|
+----+------+---+----------+-----------+
+----+------+---+----------+-----------+



In [181]:
df1 = spark.read.csv("/content/drive/MyDrive/Python Spark/test1.csv", header=True)
df2 = spark.read.csv("/content/drive/MyDrive/Python Spark/test2.csv", header=True)

In [182]:
print(df1.columns, df2.columns)

['Name', 'age', 'Experience', 'Salary'] ['Name', 'age', 'Experience', 'Salary']


In [ ]:
# 1) Union (테이블들의 컬럼이 반드시 동일)

df1.union(df2).show()
# 2) UnionByName (테이블의 컬럼이 동일하지 않아도 됨)


In [ ]:
# df1.union(df3).show()

# #union은 반드시 컬럼이 같은 테이블끼리 가능하다는 점.


In [187]:
df1.unionByName(df3, allowMissingColumns=True).show()


+-----+----+----------+------+------------+
| Name| age|Experience|Salary| Departments|
+-----+----+----------+------+------------+
| Paul|  31|        10| 30000|        NULL|
|  Tom|  30|         8| 25000|        NULL|
|Sunny|  29|         4| 20000|        NULL|
| Paul|  24|         3| 20000|        NULL|
|  Ann|  21|         1| 15000|        NULL|
|Peter|  23|         2| 18000|        NULL|
|  Ann|NULL|      NULL| 10000|Data Science|
|  Ann|NULL|      NULL|  5000|         IOT|
|  Tom|NULL|      NULL|  4000|    Big Data|
|  Ann|NULL|      NULL|  4000|    Big Data|
|  Tom|NULL|      NULL|  3000|Data Science|
|Peter|NULL|      NULL| 20000|Data Science|
|Peter|NULL|      NULL| 10000|         IOT|
|Peter|NULL|      NULL|  5000|    Big Data|
|Sunny|NULL|      NULL| 10000|Data Science|
|Sunny|NULL|      NULL|  2000|    Big Data|
+-----+----+----------+------+------------+



### 8. Window

> https://sparkbyexamples.com/pyspark/pyspark-window-functions/

In [191]:
df3 = df3.withColumn('salary',  F.col('salary').cast(T.IntegerType()))

In [192]:
# 1) window 변수 생성


from pyspark.sql import window as W


window_var = W.Window.partitionBy('Name').orderBy('salary')

# 2) window 변수가 적용된 새로운 컬럼 생성

df3.withColumn('window', F.row_number().over(window_var)).show()


+-----+------------+------+------+
| Name| Departments|salary|window|
+-----+------------+------+------+
|  Ann|    Big Data|  4000|     1|
|  Ann|         IOT|  5000|     2|
|  Ann|Data Science| 10000|     3|
|Peter|    Big Data|  5000|     1|
|Peter|         IOT| 10000|     2|
|Peter|Data Science| 20000|     3|
|Sunny|    Big Data|  2000|     1|
|Sunny|Data Science| 10000|     2|
|  Tom|Data Science|  3000|     1|
|  Tom|    Big Data|  4000|     2|
+-----+------------+------+------+



### 9. UDF(User Defined Function)

> https://velog.io/@newnew_daddy/spark05

In [200]:
# df3.show()

#UDF로 해주기

# var = 'Data Science'

# 1) 직등
#default가 string임

#pyspark.sql.functions.udf(f=None, returnType=StringType)


def str_lower(var):
  return var.lower()
udf_lower = F.udf(str_lower, returnType=T.StringType())

df3.withColumn('lower_Departments', udf_lower(F.col('Departments'))).show()


In [204]:
# 2) udf 데코레이터 사용

@F.udf(returnType=T.StringType())

def str_lower(var):
  return var.lower()

df3.withColumn('lower_Departments', str_lower(F.col('Departments'))).show()

+-----+------------+------+-----------------+
| Name| Departments|salary|lower_Departments|
+-----+------------+------+-----------------+
|  Ann|Data Science| 10000|     data science|
|  Ann|         IOT|  5000|              iot|
|  Tom|    Big Data|  4000|         big data|
|  Ann|    Big Data|  4000|         big data|
|  Tom|Data Science|  3000|     data science|
|Peter|Data Science| 20000|     data science|
|Peter|         IOT| 10000|              iot|
|Peter|    Big Data|  5000|         big data|
|Sunny|Data Science| 10000|     data science|
|Sunny|    Big Data|  2000|         big data|
+-----+------------+------+-----------------+



### 9. 기타

> https://velog.io/@newnew_daddy/spark01

> https://klefaloe4jbaezmvktwox2s3i40tudou.lambda-url.ap-northeast-2.on.aws/search

In [206]:
df3.columns

['Name', 'Departments', 'salary']

In [207]:
#컬럼 이름 일괄 변경
change_cols = ['name', 'depart', '연봉']


df3.toDF(*change_cols).show()

+-----+------------+-----+
| name|      depart| 연봉|
+-----+------------+-----+
|  Ann|Data Science|10000|
|  Ann|         IOT| 5000|
|  Tom|    Big Data| 4000|
|  Ann|    Big Data| 4000|
|  Tom|Data Science| 3000|
|Peter|Data Science|20000|
|Peter|         IOT|10000|
|Peter|    Big Data| 5000|
|Sunny|Data Science|10000|
|Sunny|    Big Data| 2000|
+-----+------------+-----+



In [208]:
# F.lit() #고정 값을 넣어주는 함수
#join시 값의 추적에 용이하덩


df3.withColumn('lit_col', F.lit('고정값')).show()



+-----+------------+------+-------+
| Name| Departments|salary|lit_col|
+-----+------------+------+-------+
|  Ann|Data Science| 10000| 고정값|
|  Ann|         IOT|  5000| 고정값|
|  Tom|    Big Data|  4000| 고정값|
|  Ann|    Big Data|  4000| 고정값|
|  Tom|Data Science|  3000| 고정값|
|Peter|Data Science| 20000| 고정값|
|Peter|         IOT| 10000| 고정값|
|Peter|    Big Data|  5000| 고정값|
|Sunny|Data Science| 10000| 고정값|
|Sunny|    Big Data|  2000| 고정값|
+-----+------------+------+-------+



In [216]:
# pyspark DataFrame -> Pandas DataFrame 으로 변경 하는 함수

pdf = df3.toPandas()

type(pdf)

pdf

,Name,Departments,salary
0,Ann,Data Science,10000
1,Ann,IOT,5000
2,Tom,Big Data,4000
3,Ann,Big Data,4000
4,Tom,Data Science,3000
5,Peter,Data Science,20000
6,Peter,IOT,10000
7,Peter,Big Data,5000
8,Sunny,Data Science,10000
9,Sunny,Big Data,2000


In [214]:
#pandas -> spark data Frame으로도 가능하다
sparkdf = spark.createDataFrame(pdf)

sparkdf.show()

+-----+------------+------+
| Name| Departments|salary|
+-----+------------+------+
|  Ann|Data Science| 10000|
|  Ann|         IOT|  5000|
|  Tom|    Big Data|  4000|
|  Ann|    Big Data|  4000|
|  Tom|Data Science|  3000|
|Peter|Data Science| 20000|
|Peter|         IOT| 10000|
|Peter|    Big Data|  5000|
|Sunny|Data Science| 10000|
|Sunny|    Big Data|  2000|
+-----+------------+------+



# pyspark sql


In [217]:
df3.show()

+-----+------------+------+
| Name| Departments|salary|
+-----+------------+------+
|  Ann|Data Science| 10000|
|  Ann|         IOT|  5000|
|  Tom|    Big Data|  4000|
|  Ann|    Big Data|  4000|
|  Tom|Data Science|  3000|
|Peter|Data Science| 20000|
|Peter|         IOT| 10000|
|Peter|    Big Data|  5000|
|Sunny|Data Science| 10000|
|Sunny|    Big Data|  2000|
+-----+------------+------+



In [221]:
df3.createOrReplaceTempView('kdt')

In [222]:
query = "SELECT * FROM kdt WHERE salary = '10000'"


spark.sql(query).show()


+-----+------------+------+
| Name| Departments|salary|
+-----+------------+------+
|  Ann|Data Science| 10000|
|Peter|         IOT| 10000|
|Sunny|Data Science| 10000|
+-----+------------+------+

